# Análise do impacto das expansões do AQE no ranking

Nesta análise vamos identificar o impacto das expansões realizadas pelo AQE nas métricas de ranking. O objetivo é identificar se utilizar mais termos na expansão pode implicar na diminuição das métricas de ranking, tendo em vista que mais documentos são recuperados. Para medir a performance do ranking vamos utilizar as queries da [base REGIS](https://github.com/Petroles/regis-collection). Para remover qualquer interferência de diferentes fatores de boosting, vamos utilizar um fator de boosting padrão de 0,1 para todos os termos expandidos.

## Carregando libs

In [ ]:
from itertools import product
import json
import yaml
import pandas as pd
import plotly.express as px

from utils.utils import get_expanded_queries, make_elasticsearch_new_aqe_queries,\
    create_new_expanded_queries, create_new_aqe_validation_dataset, create_new_aqe_metrics

## Carregando as configurações e bases dados

In [ ]:
with open("../conf/config.yaml", "r") as yamlfile:
    cfg = yaml.safe_load(yamlfile)

In [ ]:
with open("../../dados/regis/regis_queries.json", 'r') as regis_file:
    regis_queries = json.load(regis_file)

In [ ]:
regis_queries = get_expanded_queries(regis_queries)
regis_queries[:5]

## Criando queries com diferentes quantidades de expansões

Aqui vamos os conjuntos de termos expandidos para cada termo original da query, de forma que para cada termo original utilize parte de seus termos derivados.

In [ ]:
all_expanded_queries = list()
for query in regis_queries:
    new_expanded_queries = create_new_expanded_queries(query["expanded_query"])
    for num_boosts, new_expanded_query in new_expanded_queries:
        q = query.copy()
        q["expanded_query"] = new_expanded_query
        q["num_boosts"] = num_boosts
        all_expanded_queries.append(q)
all_expanded_queries[:5]

## Realizando consultas no Elasticsearch

Em posse das queries que utilizam diferentes quantidades de termos com boosting do elastic search vamos criar o dataset de validação, o qual possui informações do ground truth da base de dados REGIS.

In [ ]:
ranking_result_df = make_elasticsearch_new_aqe_queries(
    all_expanded_queries,
    cfg,
    num_docs=24
)
ranking_result_df.head()

In [ ]:
ground_truth = pd.read_csv(
    "../../dados/regis/regis_ground_truth.csv"
).rename(
    columns={"relevance": "relevance_ground_truth"}
)
ground_truth.head()

In [ ]:
validation_dataset = create_new_aqe_validation_dataset(ranking_result_df, ground_truth)
validation_dataset.head()

## Análise das consultas no Elasticsearch

Agora vamos criar as métricas para cada base de dados e quantidade de termos derivados e visualizar os resultados.

### Criando métricas

In [ ]:
metrics_df = create_new_aqe_metrics(validation_dataset)
metrics_df.head()

In [ ]:
metrics_df.to_csv("../data/aqe_expansions_metrics.csv", index=False)

### Avaliando métricas

Vamos agora avaliar as métricas. Vamos utilizar as seguintes métricas:

* ndcg - Normalized Discounted Cumulative Gain
* map - Mean Average Precision
* eval_prop - Proporção de documentos avaliados

Vejamos a performance das queries ao longo das diferentes quantidades de boost:

In [ ]:
fig = px.line(
    metrics_df,
    x="num_boosts",
    y="ndcg",
    color="query_id",
    markers=True,
    title="Performances das queries com diferentes quantidades de boosts"
)
fig.show()

Como a visualização ficou poluída, vamos verificar para cada query qual a quantidade de boosts que obteve melhor NDCG:

In [ ]:
data_viz = metrics_df.groupby(
    "query_id"
).agg({
    "ndcg": "max"
}).reset_index(
).merge(
    metrics_df, how="left", on=["query_id", "ndcg"]
).sort_values(
    ["query_id", "num_boosts"]
).drop_duplicates(
    subset="query_id", keep="first"
)
data_viz.head()

fig = px.scatter(
    data_viz,
    x="num_boosts",
    y="ndcg",
    labels={
        "num_boosts": "Número de termos",
        "ndcg": "NDCG",
    },
    hover_data=["query_id", "num_boosts", "ndcg"],
    title="Melhor número de termos por query",
    marginal_x="histogram"
)
fig.show()

É possível ver que a maior parte dos pontos está abaixo de 10 termos de boosting, apesar de quase a totalidade das queries terem mais que 10 termos. Além disso, as queries que obtiveram maior NDCG estão abaixo de 10 termos.

Vejamos qual corte de quantidade de boosting traz os melhores resultados:

In [ ]:
queries_boosts_prod = pd.DataFrame(
    product(metrics_df["query_id"].unique(), metrics_df["num_boosts"].unique()),
    columns=["query_id", "num_boosts"]
)

data_viz = queries_boosts_prod.merge(
    metrics_df, on=["query_id", "num_boosts"], how="left"
).fillna(
    method="ffill"
).groupby(
    "num_boosts"
).agg(
    mean_ndcg = ("ndcg", "mean")
).reset_index()

fig = px.line(
    data_viz,
    x="num_boosts",
    y="mean_ndcg",
    labels={
        "num_boosts": "Número de termos",
        "mean_ndcg": "NDCG médio",
    },
    markers=True,
    title="NDCG médio para cada número de termos"
)
fig.show()

Podemos ver que existe uma tendência decrescente no NDCG médio ao aumentar a quantidade de termos do AQE. Utilizar uma poda abaixo de 10 termos é algo razoável.

## Conclusão

Pudemos ver que utilizar o AQE traz um ganho maior quando apenas uma parte dos termos são utilizados.
O AQE traz entre 1 e 49 termos derivados dos termos originais e foi possível ver que utilizar menos de 10 termos traz um melhor resultado.